# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [1]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1)
        order2 = self.buy("GOOG", quantity2)

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900284  89.199593  130.451959  89.487622  127.713276   
2022-12-27  128.097488  87.332359  129.456984  88.891688  126.806960   
2022-12-28  124.166786  85.872353  129.082622  87.918349  123.999315   
2022-12-29  127.683708  88.345428  128.540773  88.757609  125.831652   
2022-12-30  127.998962  88.126938  128.018669  88.226257  125.536125   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  87.024482  128.974252  87.024482  63814900  17815000  
2022-12-27  86.940047  129.427431  88.702977  69007800  15470900  
2022-12-28  85.782968  127.742834  86.905285  85438400  17879600  
2022-12-29  86.398751  126.087782  86.438480  75703700  18280700  
2022-12-30  86.438488  126.501564  86.771210  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

result.head()

Strategy started!
Bought 1239 AAPL and 945 GOOG
Strategy finished!


,BALANCE,POSITIONS_TOTAL,EQUITY
ITERATION,,,
2018-01-02,37.157104,99962.842896,100000.000000
2018-01-03,37.157104,100774.443981,100811.601086
2018-01-04,37.157104,101190.529633,101227.686737
2018-01-05,37.157104,102505.046368,102542.203472
2018-01-08,37.157104,102537.458565,102574.615669


## <b> Backtest Statistics </b>

In [5]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 37.16
Final equity: 241907.83
---------------------------


In [7]:
bt.get_trade_statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'avg_profit': np.float64(141907.82780456543),
 'avg_profit_per_win': np.float64(141907.82780456543),
 'avg_profit_per_loss': nan,
 'median_profit': np.float64(141907.82780456543),
 'median_profit_per_win': np.float64(141907.82780456543),
 'median_profit_per_loss': nan,
 'std_dev': np.float64(nan),
 'mad_dev': np.float64(0.0),
 'downside_dev': np.float64(0.0),
 'upside_dev': np.float64(141907.82780456543),
 'skewness': np.float64(nan),
 'kurtosis': np.float64(nan),
 'mean_median_gap': np.float64(0.0),
 'avg_over_vol': np.float64(nan),
 'median_over_vol': np.float64(nan)}

In [8]:
bt.get_equity_statistics_dollar()

{'total_profit': np.float64(141907.82780456543),
 'std_dev': np.float64(4002.1716607199764),
 'mad_dev': np.float64(2424.4404661362437),
 'skewness': np.float64(-0.19347744966438507),
 'kurtosis': np.float64(3.057026293552192),
 'mean_median_gap': np.float64(-143.35560701609796),
 'max_drawdown': np.float64(-122815.60391235352)}

In [9]:
bt.get_equity_statistics_return()

{'total_return': np.float64(1.4190782780456543),
 'std_return': np.float64(0.01902567170733777),
 'mad_return': np.float64(0.014076664601513864),
 'skewness': np.float64(-0.15457324438809197),
 'kurtosis': np.float64(4.07090432388861),
 'mean_median_gap': np.float64(-0.0007787795387279695),
 'sharpe_ratio': np.float64(0.04644866453880517),
 'max_drawdown': np.float64(-0.3432091509529779)}

In [8]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,REASON_CLOSED,POSITIONS_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE
0,32b33f71-07cf-48e4-b1e8-d26c40deb24a,OPEN,Buy and Hold Trade,141907.865376,2018-01-02,None,None,241870.675186,None,None,None


In [9]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,TRADE_ID,REQUESTED_ITERATION,REQUESTED_QUANTITY,REQUESTED_PRICE,REQUESTED_FEE,REQUESTED_GROSS_TOTAL,REQUESTED_TOTAL,FILLED_ITERATION,FILLED_QUANTITY,FILLED_PRICE,FILLED_FEE,FILLED_GROSS_TOTAL,FILLED_TOTAL
0,7ff2bbcc-1333-4898-9c05-206e7978ea0a,AAPL,BUY,FILLED,304483bd-3f68-4bc5-84cf-82c4c4f34c30,2018-01-02,1239,40.341881,0.0,49983.590309,49983.590309,2018-01-02,1239,40.341881,0.0,-49983.590309,-49983.590309
1,d8a83f32-cb87-47df-abdd-72aac616f663,GOOG,BUY,FILLED,304483bd-3f68-4bc5-84cf-82c4c4f34c30,2018-01-02,945,52.888069,0.0,49979.225349,49979.225349,2018-01-02,945,52.888069,0.0,-49979.225349,-49979.225349


In [10]:
bt.strategy.positions

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.7144165039)},
 'GOOG': {'quantity': 945, 'value': np.float64(83279.95628356934)}}

In [13]:
bt.result.EQUITY

ITERATION
2018-01-02    100000.000000
2018-01-03    100811.601086
2018-01-04    101227.686737
2018-01-05    102542.203472
2018-01-08    102574.615669
                  ...      
2022-12-23    245277.223747
2022-12-27    241279.024788
2022-12-28    235029.178406
2022-12-29    241723.701454
2022-12-30    241907.827805
Name: EQUITY, Length: 1259, dtype: float64